Подготовим Dataset:

In [20]:
from langchain_community.document_loaders import PyPDFLoader

In [21]:
data_folder = './dataset/'

In [22]:
pdf = PyPDFLoader(
    file_path = data_folder+"Energetic_strategy.pdf",
    extract_images= False
)

In [32]:
pdf.load()[1].page_content

' \n  \nУТВЕРЖДЕНА  \nраспоряжением Правительства  \nРоссийской Федерации  \nот 9 июня 2020 г.  № 1523 -р \n \n \n \n \n  \nЭНЕРГЕТ ИЧЕСКАЯ СТРАТЕГИЯ   \nРоссийской Федерации на период до 2035  года  \n \n \nI. Общие положения  \n \nНастоящая Стратегия разработана в соответствии с Федеральным \nзаконом "О стратегическом планировании в Российской Федерации ", \nПравилами разработки, корректировки, осуществл ения мониторинга и \nконтроля реализации отраслевых документов стратегического \nпланирования Российской Федерации по вопросам, находящимся в \nведении Правительства Российской Федерации, утвержденными \nпостановлением Правительства Российской Федерации от 29  октября 2015  г. \n№ 1162 "Об утверждении Правил разработки, корректировки, \nосуществления мониторинга и контроля реализации отраслевых \nдокументов стратегического планирования Российской Федерации  \nпо вопросам, находящимся в ведении Правительства Российской \nФедера ции", и является межотраслевой стратегией для совокупност

# Разабьём текст и получим векторное хранилище:

Разабьём текс на сигменты:

In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n \n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    chunk_size=1000, 
    chunk_overlap=128)

fragments = splitter.create_documents([i.page_content for i in pdf.load()])

In [92]:
fragments

[Document(page_content='ПРАВИТЕЛЬСТВО   РОССИЙСКОЙ   ФЕДЕРАЦИИ  \n \nР А С П О Р Я Ж Е Н И Е  \n \nот 9 июня 2020 г.  № 1523 -р   \n \nМОСКВА'),
 Document(page_content='1. Утвердить прилагаемую Энергетическую стратегию Российской \nФедерации на период до 2035 года (далее  - Стратегия).  \n2. Федеральным органам исполн ительной власти руководствоваться \nположениями Стратегии при разработке и корректировке государственных \nпрограмм Российской Федерации и иных документов стратегического \nпланирования.  \n3. Рекомендовать органам государственной власти субъектов \nРоссийской Федера ции руководствоваться положениями Стратегии при \nразработке и корректировке государственных программ субъектов \nРоссийской Федерации и иных документов стратегического планирования.  \n4. Минэнерго России совместно с заинтересованными федеральными \nорганами испол нительной власти:  \nпредставить в 6 -месячный срок в Правительство Российской \nФедерации проект плана мероприятий по реализации Стратегии;  \nоб

Подключим Embeddings из yandex chain:

In [1]:
from yandex_chain import YandexEmbeddings
from langchain.vectorstores import FAISS

In [82]:
embeddings_model = YandexEmbeddings(config="config.json")

In [98]:
vectorstore = FAISS.from_texts(
    [
        i.page_content
        for i in fragments[:100]
    ], 
    embedding=embeddings_model
    )
retriever = vectorstore.as_retriever()

Проверим работу:

In [103]:
from yandex_chain import YandexLLM

In [106]:
llm = YandexLLM(config="config.json")

In [109]:
from langchain_core.prompts import ChatPromptTemplate

In [112]:
def query(text):
    vect = retriever.invoke(text)
    temple = ChatPromptTemplate([
        ("system", 'Дай ответ на вопрос исходи из {vect}'),
        ("user", '{text}')
    ])
    out = llm.invoke(
        temple.invoke({
            'vect':vect,
            'text':text
        })
    )
    return out, vect

In [114]:
print(query(input()))

('9 июня 2020 г.', [Document(page_content='5. Признать утратившим силу распоряжение Правительства \nРоссийской Ф едерации от 13 ноября  2009 г. №  1715 -р (Собрание \nзаконодательств а Российской Федерации, 2009, №  48, ст.  5836).'), Document(page_content='ПРАВИТЕЛЬСТВО   РОССИЙСКОЙ   ФЕДЕРАЦИИ  \n \nР А С П О Р Я Ж Е Н И Е  \n \nот 9 июня 2020 г.  № 1523 -р   \n \nМОСКВА'), Document(page_content='1. Утвердить прилагаемую Энергетическую стратегию Российской \nФедерации на период до 2035 года (далее  - Стратегия).  \n2. Федеральным органам исполн ительной власти руководствоваться \nположениями Стратегии при разработке и корректировке государственных \nпрограмм Российской Федерации и иных документов стратегического \nпланирования.  \n3. Рекомендовать органам государственной власти субъектов \nРоссийской Федера ции руководствоваться положениями Стратегии при \nразработке и корректировке государственных программ субъектов \nРоссийской Федерации и иных документов стратегического планирован